In [2]:
import operator
from typing import Annotated, TypedDict
from langchain_community.chat_models import ChatOllama
from langgraph.graph import StateGraph, END
from IPython.display import display, Markdown

# --- 1. Define the Graph State ---
- This defines the "schema" of the state that will be passed between nodes.

In [3]:

class GraphState(TypedDict):
    """
    Represents the state of our graph.

    Attributes:
        question: The user's initial question.
        response: The accumulated response from the LLMs.
    """
    question: str
    response: Annotated[str, operator.add] # Use operator.add to append responses

In [4]:
# --- 2. Initialize Local LLMs ---
# Assuming you have Ollama running and these models downloaded:
# ollama run llama3:8b

try:
    llm_1 = ChatOllama(model="qwen3:0.6b", temperature=0) # Often a good general-purpose lightweight model
    llm_2 = ChatOllama(model="qwen3:0.6b", temperature=0) # Another popular lightweight choice
    llm_3 = ChatOllama(model="qwen3:0.6b", temperature=0) # Very small, good for basic tasks
except Exception as e:
    print(f"Error initializing Ollama models. Make sure Ollama is running and models are downloaded.")
    print(f"You can download models with: ollama run qwen3:0.6b")
    print(f"Error: {e}")
    exit()

In [5]:
# --- 3. Define Nodes (Functions for each LLM) ---

def llm_node_1(state: GraphState):
    """
    First LLM processing node.
    """
    print("---NODE 1: Llama3 Processing---")
    question = state["question"]
    # Simulate some processing by the LLM
    llm_response = llm_1.invoke(f"""Process this: {question}. Add a brief introduction. 
                                Start your final response with text **'Response 1:'** """)
    
    return {"response": f"Node 1 Qwen's take: {llm_response.content}\n"}

def llm_node_2(state: GraphState):
    """
    Second LLM processing node.
    """
    print("---NODE 2: Mistral Processing---")
    question = state["question"]
    current_response = state.get("response", "")
    #current_response = current_response.split("Response 1:")[-1]
    # Simulate more processing based on the question and previous response
    llm_response = llm_2.invoke(f"""Elaborate on: '{question}'. Consider previous output: '{current_response}' 
                                Start your final response with text **'Response 2:'** """)

    return {"response": f"Node 2 Qwen's take: {llm_response.content}\n"}


def llm_node_3(state: GraphState):
    """
    Third LLM processing node.
    """
    print("---NODE 3: TinyLlama Processing---")
    question = state["question"]
    current_response = state.get("response", "")
    # Simulate a final touch or summary
    llm_response = llm_3.invoke(f"""Summarize the main point of '{question}' after seeing this: '{current_response}'.
                                Start your final response with text **'Response 3:'** """)

    return {"response": f"Llama3's take: {llm_response.content}\n"}

In [6]:
# --- 4. Build the LangGraph Workflow ---

workflow = StateGraph(GraphState)

# Add nodes to the workflow
workflow.add_node("QwenNode1", llm_node_1)
workflow.add_node("QwenNode2", llm_node_2)
workflow.add_node("QwenNode3", llm_node_3)

# Define the sequence of execution
workflow.set_entry_point("QwenNode1") # Start with Llama3
workflow.add_edge("QwenNode1", "QwenNode2") # After Llama3, go to Mistral
workflow.add_edge("QwenNode2", "QwenNode3") # After Mistral, go to TinyLlama
workflow.add_edge("QwenNode3", END) # After TinyLlama, end the graph

# Compile the graph
app = workflow.compile()

In [7]:
user_question = "Explain the concept of masked language modeling task in simple language."
for s in app.stream({"question": user_question, "response": ""}):
    node_name = list(s.keys())[0]
    display(Markdown(s[node_name]["response"]))
    print("="*25)

---NODE 1: Llama3 Processing---


Node 1 Qwen's take: <think>
Okay, the user wants me to explain masked language modeling tasks in simple language, add a brief introduction, and start with "Response 1:."

First, I need to make sure I understand what masked language modeling is. From what I remember, it's a technique used in machine learning where the model is trained to understand and generate text that's similar to the input. But how to simplify that?

The user mentioned "masked language modeling task," so I should explain that. Maybe start with a simple example, like a chatbot that can understand and generate text. Then mention that the model is trained to understand the context, not just the words. Also, include that it's used in various fields like AI, natural language processing, etc.

Wait, the user wants a brief introduction first. So maybe something like: "In masked language modeling, the model learns to understand and generate text by being trained on a large dataset of text. This allows it to understand context and generate text that's similar to the input."

I need to check if that's all. Make sure the response starts with "Response 1:" and is in simple language. Avoid technical jargon. Let me put it all together.
</think>

**Response 1:**  
Masked language modeling is a technique used in machine learning where the model learns to understand and generate text by being trained on a large dataset of text. This allows it to understand context and generate text that's similar to the input. It’s used in various fields like AI, natural language processing, and language modeling to create text that mimics human speech.


---NODE 2: Mistral Processing---


Node 2 Qwen's take: <think>
Okay, the user wants me to elaborate on "masked language modeling task in simple language." They also mentioned considering previous output, which I did. Now, I need to make sure my response is simple and starts with "Response 2:."

First, I'll explain masked language modeling in a basic way. Maybe use an example like a chatbot that can understand and generate text. Then mention that the model learns by being trained on a large dataset. I should keep it straightforward and avoid any technical terms. Let me check if that covers all the points and starts with "Response 2:".
</think>

**Response 2:**  
Masked language modeling is when a machine learning model learns to understand and generate text by being trained on a large dataset of text. This helps it understand context and generate text that's similar to the input. It's used in AI, language processing, and other fields to create text that mimics human speech.


---NODE 3: TinyLlama Processing---


Llama3's take: <think>
Okay, the user wants me to summarize the main point of "masked language modeling task in simple language" after seeing the previous response. Let me check the previous response to make sure I'm on the right track.

In the first response, I explained masked language modeling as a technique where a model learns to understand and generate text by being trained on a large dataset. The user wants a summary in simple language, so I need to rephrase that into a brief explanation. Let me make sure to mention that it's used in AI, natural language processing, etc., and that it helps with generating text similar to the input. I should start with "Response 3:" to match the previous structure. Let me put that together clearly.
</think>



In [10]:
for s in app.stream({"question": user_question, "response": ""}):
    node_name = list(s.keys())[0]
    #print(s[node_name]["response"])
    display(Markdown(s[node_name]["response"]))
    print("---")

---NODE 1: Llama3 Processing---


Node 1 Qwen's take: <think>
Okay, the user wants me to explain masked language modeling tasks in simple language, add a brief introduction, and start with "Response 1:."

First, I need to make sure I understand what masked language modeling is. From what I remember, it's a technique used in machine learning where the model is trained to understand and generate text that's similar to the input. But how to simplify that?

The user mentioned "masked language modeling task," so I should explain that. Maybe start with a simple example, like a chatbot that can understand and generate text. Then mention that the model is trained to understand the context, not just the words. Also, include that it's used in various fields like AI, natural language processing, etc.

Wait, the user wants a brief introduction first. So maybe something like: "In masked language modeling, the model learns to understand and generate text by being trained on a large dataset of text. This allows it to understand context and generate text that's similar to the input."

I need to check if that's all. Make sure the response starts with "Response 1:" and is in simple language. Avoid technical jargon. Let me put it all together.
</think>

**Response 1:**  
Masked language modeling is a technique used in machine learning where the model learns to understand and generate text by being trained on a large dataset of text. This allows it to understand context and generate text that's similar to the input. It’s used in various fields like AI, natural language processing, and language modeling to create text that mimics human speech.


---
---NODE 2: Mistral Processing---


Node 2 Qwen's take: <think>
Okay, the user wants me to elaborate on "masked language modeling task in simple language." They also mentioned considering previous output, which I did. Now, I need to make sure my response is simple and starts with "Response 2:."

First, I'll explain masked language modeling in a basic way. Maybe use an example like a chatbot that can understand and generate text. Then mention that the model learns by being trained on a large dataset. I should keep it straightforward and avoid any technical terms. Let me check if that covers all the points and starts with "Response 2:".
</think>

**Response 2:**  
Masked language modeling is when a machine learning model learns to understand and generate text by being trained on a large dataset of text. This helps it understand context and generate text that's similar to the input. It's used in AI, language processing, and other fields to create text that mimics human speech.


---
---NODE 3: TinyLlama Processing---


Llama3's take: <think>
Okay, the user wants me to summarize the main point of "masked language modeling task in simple language" after seeing the previous response. Let me check the previous response to make sure I'm on the right track.

In the first response, I explained masked language modeling as a technique where a model learns to understand and generate text by being trained on a large dataset. The user wants a summary in simple language, so I need to rephrase that into a brief explanation. Let me make sure to mention that it's used in AI, natural language processing, etc., and that it helps with generating text similar to the input. I should start with "Response 3:" to match the previous structure. Let me put that together clearly.
</think>



---
